In [1]:
import torch
import io
import os
import json
from botorch.models import SingleTaskGP
from botorch.fit import fit_gpytorch_mll
from gpytorch.mlls import ExactMarginalLogLikelihood
import asyncio
dtype = torch.float
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def x_to_filename(x):
    return f'{x[0]}_{x[1]}_{x[2]}_{x[3]}.json'

async def evaluate_profile(x):
    filename=x_to_filename(x)
    display(filename)
    if not filename in os.listdir("case/results"):
        config=json.dumps({
            'p':x[0],
            'm':x[1],
            't':x[2],
            'a':x[3]
        })
        p=await asyncio.create_subprocess_shell(f'cd case; ./evaluate_airfoil.sh \'{config}\'',stdout=asyncio.subprocess.DEVNULL)
        await p.wait()
        #os.system(f'cd case; ./evaluate_airfoil.sh \'{config}\'')
    with open("case/results/"+filename) as file:
        result=json.load(file)
        display(result)
        return [result['C_L'],result['C_L']/result['C_D']]

await evaluate_profile([0.42,0.05,0.18,11])

'0.42_0.05_0.18_11.json'

{'C_L': 1.26294,
 'C_D': 0.141091,
 'C_N': 1.27673,
 'C_T': -0.05064858,
 'angle': 0.1919862177193889}

[1.26294, 8.951244232445726]

In [2]:
x_lims=[(0.1,0.9),(-0.2,0.2),(0.05,0.3),(0.01,35)]
initial_X = torch.rand(6,4)
for d in range(4):
    initial_X[:,d]=x_lims[d][0]+initial_X[:,d]*(x_lims[d][1]-x_lims[d][0])
y=await asyncio.gather(*[evaluate_profile(x.tolist()) for x in initial_X])

'0.8655616641044617_-0.11298897117376328_0.06051531806588173_33.45952224731445.json'

'0.1629708707332611_-0.19246254861354828_0.06699815392494202_2.2169487476348877.json'

'0.8381821513175964_0.18810896575450897_0.10013279318809509_13.680347442626953.json'

'0.7387102842330933_-0.17652557790279388_0.07220277190208435_15.102472305297852.json'

'0.5877008438110352_0.059707775712013245_0.18491235375404358_24.791046142578125.json'

'0.27554646134376526_-0.034876495599746704_0.2402607649564743_25.372297286987305.json'

The latest version of Julia in the `release` channel is 1.8.4+0.x64.linux.gnu. You currently have `1.8.3+0.x64.linux.gnu` installed. Run:

  juliaup update

to install Julia 1.8.4+0.x64.linux.gnu and update the `release` channel to that version.
The latest version of Julia in the `release` channel is 1.8.4+0.x64.linux.gnu. You currently have `1.8.3+0.x64.linux.gnu` installed. Run:

  juliaup update

to install Julia 1.8.4+0.x64.linux.gnu and update the `release` channel to that version.
The latest version of Julia in the `release` channel is 1.8.4+0.x64.linux.gnu. You currently have `1.8.3+0.x64.linux.gnu` installed. Run:

  juliaup update

to install Julia 1.8.4+0.x64.linux.gnu and update the `release` channel to that version.
The latest version of Julia in the `release` channel is 1.8.4+0.x64.linux.gnu. You currently have `1.8.3+0.x64.linux.gnu` installed. Run:

  juliaup update

to install Julia 1.8.4+0.x64.linux.gnu and update the `release` channel to that version.
The latest versi

CancelledError: 

In [15]:

train_X=initial_X
print(train_X)
Y = 1 - (train_X - 0.5).norm(dim=-1, keepdim=True)  # explicit output dimension
Y += 0.1 * torch.rand_like(Y)
train_Y = (Y - Y.mean()) / Y.std()
print(train_X)
print(train_Y)

gp = SingleTaskGP(train_X, train_Y)
mll = ExactMarginalLogLikelihood(gp.likelihood, gp)
fit_gpytorch_mll(mll)
test_X=torch.rand(1,2)
print(test_X,": ",gp.posterior(test_X).mean)
from botorch.acquisition import UpperConfidenceBound

UCB = UpperConfidenceBound(gp, beta=0.1)
from botorch.optim import optimize_acqf

bounds = torch.stack([torch.zeros(2), torch.ones(2)])
candidate, acq_value = optimize_acqf(
    UCB, bounds=bounds, q=1, num_restarts=5, raw_samples=20,
)

tensor([[ 7.2817e-01, -3.4658e-02,  1.3854e-01,  2.5941e+01],
        [ 6.9175e-01,  1.1860e-01,  9.8697e-02,  3.2845e+01],
        [ 8.0765e-01,  1.5980e-01,  2.4200e-01,  1.9580e+01],
        [ 1.6823e-01,  1.7437e-01,  1.7050e-01,  1.6646e+01],
        [ 7.1448e-01, -1.7571e-02,  1.1489e-01,  2.4194e+01],
        [ 8.5181e-01, -1.8925e-01,  2.2223e-01,  2.6894e+01]])
tensor([[ 7.2817e-01, -3.4658e-02,  1.3854e-01,  2.5941e+01],
        [ 6.9175e-01,  1.1860e-01,  9.8697e-02,  3.2845e+01],
        [ 8.0765e-01,  1.5980e-01,  2.4200e-01,  1.9580e+01],
        [ 1.6823e-01,  1.7437e-01,  1.7050e-01,  1.6646e+01],
        [ 7.1448e-01, -1.7571e-02,  1.1489e-01,  2.4194e+01],
        [ 8.5181e-01, -1.8925e-01,  2.2223e-01,  2.6894e+01]])
tensor([[-0.2845],
        [-1.4803],
        [ 0.8303],
        [ 1.3566],
        [ 0.0223],
        [-0.4443]])


/home/olep/.local/lib/python3.10/site-packages/botorch/models/gpytorch.py:113: UserWarning: The model inputs are of type torch.float32. It is strongly recommended to use double precision in BoTorch, as this improves both precision and stability and can help avoid numerical errors. See https://github.com/pytorch/botorch/discussions/1444
  warnings.warn(


RuntimeError: Sizes of tensors must match except in dimension 0. Expected size 4 but got size 2 for tensor number 1 in the list.